# Mini Project 1 — Module 3: Database Systems  
### Customer–Product Data Integration System

This project demonstrates how customer and product data from two different systems 
can be integrated into a centralized database that supports analytical queries 
and reporting.

### **Problem Overview**
You are required to design and implement a database that integrates:

- **Customer Data System**  
  Contains customer IDs, names, addresses, and contact details.

- **Product Data System**  
  Provided via CSV or external API, containing product names, prices, categories, and stock levels.

Additionally, you must:
- Integrate both datasets into a unified schema
- Create and populate database tables
- Demonstrate SQL queries retrieving combined insights
- Validate data integrity
- Produce a summary report of the integration process

The implemented solution uses:
- **Python ETL scripts**  
- **Supabase PostgreSQL database**  
- **DBDocs ERD**  
- **SQL insights queries**  


# Database Schema (ERD)

The integrated database schema includes four main tables:

- **customers**
- **products**
- **orders**
- **order_items**

### ERD Online (DBDocs)
Full schema and relationships:  
https://dbdocs.io/gabrielndunda/Customer-Product-Data-Integration-System?view=relationships

### DBML Schema
```dbml
Table customers {
  customer_id int [pk]
  first_name varchar
  last_name varchar
  email varchar
  phone varchar
  address_line1 varchar
  address_line2 varchar
  city varchar
  country varchar
  created_at timestamp
}

Table products {
  product_id int [pk]
  product_name varchar
  price decimal
  stock_quantity int
  source_system varchar
  created_at timestamp
}

Table orders {
  order_id int [pk]
  customer_id int [ref: > customers.customer_id]
  order_date date
  total_amount decimal
}

Table order_items {
  order_item_id int [pk]
  order_id int [ref: > orders.order_id]
  product_id int [ref: > products.product_id]
  quantity int
  unit_price decimal
  subtotal decimal
}


In [3]:
## 🔵 **Notebook Cell 3 — Configure Environment & DB Connection**
import os
from dotenv import load_dotenv
import psycopg2
import pandas as pd
from psycopg2.extras import RealDictCursor

# Load .env file (must contain SUPABASE_DB_URL)
load_dotenv()
SUPABASE_DB_URL = os.getenv("SUPABASE_DB_URL")

if SUPABASE_DB_URL is None:
    raise ValueError("SUPABASE_DB_URL is missing. Add it to your .env file.")

def run_query(query, params=None):
    conn = psycopg2.connect(SUPABASE_DB_URL)
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(query, params)
            return pd.DataFrame(cur.fetchall())
    finally:
        conn.close()

# Test connection
run_query("SELECT NOW();")


,now
0,2025-12-05 11:45:07.697408+00:00


# Working Implementation: Data Integration

This solution integrates data from the two systems:

### **1. Customer Data Integration**
- Generated using a synthetic Kenyan name generator  
- Includes names, gender, county → mapped to city  
- Contact details from Faker library  
- Inserted into the **customers** table  

### **2. Product Data Integration (from CSV)**
- CSV contains 2,000 product records  
- Columns include name, price, quantity, category  
- Loaded into Python using Pandas  
- Inserted into the **products** table  

### **3. Linking Tables through Orders**
Synthetic orders and order_items link:
- customers → orders → order_items → products  

This creates a fully integrated structure suitable for analytics.


In [4]:
# View first few customers
run_query("SELECT * FROM customers LIMIT 10;")

,customer_id,first_name,last_name,email,phone,address_line1,address_line2,city,country,created_at
0,1,Timothy,Kipruto,timothy.kipruto1@example.com,001-989-394-9622x1530,18 Garcia Spurs,Suite 214,Uasin Gishu,Kenya,2025-12-05 10:28:14.513648
1,2,Vanessa,Otieno,vanessa.otieno2@example.com,490-450-8227x10463,134 Nelson Loop,Apt. 108,Nairobi,Kenya,2025-12-05 10:28:14.513749
2,3,Jack,Wambui,jack.wambui3@example.com,371.877.0373x28840,7 Harmon Village,Apt. 609,Kirinyaga,Kenya,2025-12-05 10:28:14.513837
3,4,Carol,Cherono,carol.cherono4@example.com,634.256.7064,32 Scott Place,Suite 454,Kericho,Kenya,2025-12-05 10:28:14.513927
4,5,Samuel,Nyanchoka,samuel.nyanchoka5@example.com,555-260-0450x20471,61 Anthony Fields,Apt. 719,Kisii,Kenya,2025-12-05 10:28:14.514001
5,6,Connor,Kibet,connor.kibet6@example.com,+1-450-522-9274x6642,169 Edwards Flats,Suite 094,Elgeyo Marakwet,Kenya,2025-12-05 10:28:14.514084
6,7,Nicholas,Wambui,nicholas.wambui7@example.com,(644)211-2333x38119,6 Lam Villages,Suite 794,Laikipia,Kenya,2025-12-05 10:28:14.514167
7,8,Debra,Mutembei,debra.mutembei8@example.com,994.478.2128,44 King Dale,Apt. 342,Embu,Kenya,2025-12-05 10:28:14.514251
8,9,Shane,Chepkoech,shane.chepkoech9@example.com,723-494-3954,75 Mckenzie Station,Apt. 018,Kericho,Kenya,2025-12-05 10:28:14.514332
9,10,Michael,Omondi,michael.omondi10@example.com,(676)360-6051,127 Heather Terrace,Suite 059,Migori,Kenya,2025-12-05 10:28:14.514401


In [5]:
# View products
run_query("SELECT * FROM products LIMIT 10;")


,product_id,product_name,price,stock_quantity,source_system,created_at
0,1,"Smart TV 43"" Deluxe",2786.54,114,CSV,2025-12-05 10:28:14.505516
1,2,"Smart TV 43"" Value",4248.12,15,CSV,2025-12-05 10:28:14.505729
2,3,Basmati Rice 5kg Value,6040.09,287,CSV,2025-12-05 10:28:14.505731
3,4,Juice 1L Pro,4519.63,142,CSV,2025-12-05 10:28:14.505734
4,5,Milk 1L 2024 Edition,2814.82,110,API,2025-12-05 10:28:14.505736
5,6,"Smart TV 43"" Limited",1012.33,433,API,2025-12-05 10:28:14.505738
6,7,Tusker Lager 500ml Mini,482.33,235,CSV,2025-12-05 10:28:14.505739
7,8,Kitenge Dress Value,2967.12,321,API,2025-12-05 10:28:14.505745
8,9,Mineral Water 1L Deluxe,742.07,338,CSV,2025-12-05 10:28:14.505748
9,10,Dining Table Mini,2366.27,51,API,2025-12-05 10:28:14.505750


# SQL Queries Demonstrating Integration & Insights
These queries show how customer and product data are combined after integration.


## Query 1 — Customer Orders with Product Details

In [6]:
q1 = """
SELECT 
    o.order_id,
    c.first_name,
    c.last_name,
    p.product_name,
    oi.quantity,
    oi.unit_price,
    oi.subtotal
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON oi.order_id = o.order_id
JOIN products p ON p.product_id = oi.product_id
LIMIT 20;
"""
run_query(q1)


,order_id,first_name,last_name,product_name,quantity,unit_price,subtotal
0,1,Russell,Mutembei,Bluetooth Speaker Classic,2,3914.53,7829.06
1,2,Kelly,Wanjala,Dining Table Mini,4,9281.05,37124.20
2,3,Zachary,Chebet,Tusker Lager 500ml Classic,2,3671.59,7343.18
3,4,Sarah,Ebei,Kitenge Dress Value,5,7680.10,38400.50
4,5,Alexandra,Achieng,Maize Flour 2kg Mini,2,2856.59,5713.18
5,6,Mary,Baya,Laptop Slim Mini,1,6219.03,6219.03
6,7,Tony,Mwangi,Sugar 2kg Pro,2,5799.98,11599.96
7,8,Jeffrey,Baya,Maize Flour 2kg Pro,5,8560.03,42800.15
8,9,Susan,Koech,Sugar 2kg Pro,1,2242.89,2242.89
9,10,James,Kassim,Basmati Rice 5kg Mini,3,9133.10,27399.30


## Query 2 — Total Value of Orders per Customer

In [8]:
q2 = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    SUM(o.total_amount) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC
LIMIT 20;
"""
run_query(q2)


,customer_id,first_name,last_name,total_spent
0,1084,Kevin,Kiplangat,141890.49
1,304,Caleb,Moraa,130135.49
2,805,Matthew,Kamau,109607.18
3,915,Anthony,Mutiso,108100.58
4,335,Jennifer,Bakari,107181.63
5,511,Ann,Mukhwana,99891.34
6,205,Stephen,Odhiambo,97887.90
7,983,Heather,Wario,93371.20
8,430,Tiffany,Nyakundi,90981.52
9,1552,John,Kimani,90063.98


## Query 3 — Filter Products by Price Range

In [9]:
min_price = 100
max_price = 5000

q3 = """
SELECT *
FROM products
WHERE price BETWEEN %s AND %s
LIMIT 50;
"""
run_query(q3, (min_price, max_price))


,product_id,product_name,price,stock_quantity,source_system,created_at
0,1,"Smart TV 43"" Deluxe",2786.54,114,CSV,2025-12-05 10:28:14.505516
1,2,"Smart TV 43"" Value",4248.12,15,CSV,2025-12-05 10:28:14.505729
2,4,Juice 1L Pro,4519.63,142,CSV,2025-12-05 10:28:14.505734
3,5,Milk 1L 2024 Edition,2814.82,110,API,2025-12-05 10:28:14.505736
4,6,"Smart TV 43"" Limited",1012.33,433,API,2025-12-05 10:28:14.505738
5,7,Tusker Lager 500ml Mini,482.33,235,CSV,2025-12-05 10:28:14.505739
6,8,Kitenge Dress Value,2967.12,321,API,2025-12-05 10:28:14.505745
7,9,Mineral Water 1L Deluxe,742.07,338,CSV,2025-12-05 10:28:14.505748
8,10,Dining Table Mini,2366.27,51,API,2025-12-05 10:28:14.505750
9,12,Office Chair Deluxe,2706.43,479,CSV,2025-12-05 10:28:14.505753


## Query 4 — Customers with High-Value Orders

In [10]:
q4 = """
SELECT 
    o.order_id,
    c.first_name,
    c.last_name,
    o.total_amount
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.total_amount > 8000
ORDER BY o.total_amount DESC;
"""
run_query(q4)


,order_id,first_name,last_name,total_amount
0,172,Mallory,Mwangi,49991.10
1,833,Alex,Mukasa,49906.05
2,69,Melanie,Koech,49684.00
3,1194,Tiffany,Nyakundi,49576.90
4,16,Jean,Ole,49460.80
...,...,...,...,...
1263,589,Vincent,Wekesa,8019.87
1264,951,Caitlyn,Barasa,8013.64
1265,961,Angela,Kimani,8012.70
1266,978,Jessica,Sankale,8012.64


# 🔍 Data Validation

To ensure consistency:

#### Foreign key relationships validated  
- Every order references an existing customer  
- Every order item references an existing order  
- Every order item references an existing product  

#### Record counts validated  
- 2,000 customers  
- 2,000 products  
- 2,000 orders  
- 2,000 order items  

#### No missing or inconsistent foreign keys  

These checks confirm the database is correctly integrated and queryable.


# Summary of Integration Process

This notebook demonstrates a complete data integration workflow:

#### 1Designed a relational schema (ERD + DBML)  
#### Set up a cloud PostgreSQL database (Supabase)  
#### Imported customer and product data from different systems  
#### Linked the datasets through orders and order items  
#### Executed analytical SQL queries combining multiple sources  
#### Validated consistency and structure  

The system now supports:
- Customer–order analytics  
- Product sales reporting  
- Revenue calculations  
- CSV-to-database verification  

This completes all deliverables for **Mini Project 1 — Module 3: Database Systems**.
